# <CENTER>CHECK PCA TABLES<CENTER>

This script is checking PCA tables which are used for creating Power BI reports:
- DW.CONTACT_CAMPAIGN
- CS.NON_CARD_CONTACT


- DWS.AC_CAMPAIGN
- DWS.AC_SEND
- DWS.AC_OPEN
- DWS.AC_CLICK
- DWS.AC_UNSUBSCRIBE


- CS.PCA_CAMPAIGN_TRACKER
- CS.PCA_CAMPAIGN_ID_COUNTRY 
- CS.PCA_CAMPAIGN_LANDING
- CS.PCA_CAMPAIGN_DETAILS
- CS.PCA_CAMP_ID
- CS.PCA_CHEETAH_CH_MATCH
- CS.PCA_TRANSACTION_AGG
- CS.PCA_EMAIL_ENGAGEMENT
- CS.PCA_EMAIL_RESPONSE_AGG
- CS.PCA_CUST_SUMMARY


- CS.PCA_CAMPAIGN_BRAND **
- CS.PCA_BRAND_TRANSACTIONS
- CS.PCA_BRAND_TRANSACTIONS_SEGMENTATION **
- CS.PCA_BRAND_OR_CATEGORY_PRODUCT_TABLE_MATCHED
- CS.PCA_FEATURED_PRODUCT_TABLE_MATCHED


- CS.PCA_EMAIL_CH_NON_CARD_ONLY
- CS.PCA_TRANSACTION_AGG_NON_CARD **


- CS.PCA_OFFER_PERFORMANCE_FINAL_SEGMENTATION **
- CS.PCA_FINAL_TABLE_SEGMENTATION **
- CS.PCA_FINAL_TABLE

- CS.PCA_DM_FINAL_TABLE_SEGMENTATION (for DM)
- CS.PCA_DM_FINAL_TABLE (for DM)

Note: you could delete tables marked with ** if you were using Jo's troubleshooting script. Data need to be deleted in case there was a change in the Nampara script (or if PCA info hasn't been loaded correctly), so the change could take effect on data populating Nampara tables.

Note 2: CS.PCA_CAMPAIGN_ID_COUNTRY only picks up campaigns sent in the last 30 days. As a rule you probably don’t want to delete from CS.PCA_CAMPAIGN_ID_COUNTRY unless you manually insert it back in after deleting or the campaign was sent in the last 30 days.

Note 3: PCA tables are UPDATED after running the tracker script and the Nampara script, which are run automatically every Tuesday and Friday (at 3pm and 4pm respectively), IF THE CAMPAIGN IS LIVE (i.e. if it's analysis period end date is within last 7 days). Then, updated Power BI results are available following the manual refresh of Power BI reports, which is on Wednesdays and Mondays at 9am.

The checks below are useful if Power BI results are missing or look incorrect.

## IMPORT AND CONNECT

In [1]:
%load_ext sql

import ibm_db_sa
import pandas as pd

# Return Pandas DataFrames instead of regular result sets
%config SqlMagic.autopandas=True

# Display all rows/columns of Pandas DataFrame
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
### Read your credentials from .env file
import os
from dotenv import load_dotenv
load_dotenv(r'C:\Users\bonnie.li\Desktop\Credentials.env')
username = os.getenv('ABACUS_USERNAME') 
password = os.getenv('ABACUS_PASSWORD')

credentials = f'{username}:{password}'
%sql ibm_db_sa://$credentials@abacusproddbcluster:50000/BCUDB
#sql ibm_db_sa://username:password@hostname:port/database_name

'Connected: libonnie@BCUDB'

## ENTER CAMPAIGN INFORMATION

In [3]:
campaign_code = 'C000020610'

## CHECK CONTACT HISTORY - DONOTDELETE

- Card

In [4]:
%sql SELECT * FROM DW.CONTACT limit 1

 * ibm_db_sa://libonnie:***@abacusproddbcluster:50000/BCUDB
Done.


,account_number,cell_id,cell_code,package_id,contact_date_time,contact_date,contact_status_name,campaign_code,treatment_code,control_treatment_flag,offer_code,segment_name,spend_band,child_number,child_dob_baby_due,dob_due_date_flag,welcome_pack_no,stage_pack_no,exception_reason_code
0,1013775936,1640417,A001640417,205589,2021-09-29 00:09:01,2021-09-29,Delivered,C000012419,102798694,0,000072024,None,None,None,None,None,None,None,None


In [5]:
%%sql date <<
    SELECT CONTACT_DATE, CONTACT_DATE_TIME, CONTROL_TREATMENT_FLAG, COUNT(*) AS mailed_control 
    FROM DW.CONTACT
    WHERE CAMPAIGN_CODE = :campaign_code 
    GROUP BY CONTACT_DATE, CONTACT_DATE_TIME, CONTROL_TREATMENT_FLAG
    ORDER BY CONTACT_DATE_TIME DESC

 * ibm_db_sa://libonnie:***@abacusproddbcluster:50000/BCUDB
Done.
Returning data to local variable date


In [6]:
date

,contact_date,contact_date_time,control_treatment_flag,mailed_control
0,2022-08-08,2022-08-08 10:48:45,0,314745
1,2022-08-08,2022-08-08 10:48:45,1,31470
2,2022-07-21,2022-07-21 14:43:24,0,9417
3,2022-07-21,2022-07-21 14:43:24,1,941


In [7]:
contact_date = date['contact_date'][0].strftime('%Y-%m-%d')
contact_date

'2022-08-08'

- Non-card

In [ ]:
%%sql non_card_date <<
    SELECT CONTACT_DATE, COUNT(*) AS mailed_control 
    FROM CS.NON_CARD_CONTACT
    WHERE CAMPAIGN_CODE = :campaign_code 
    GROUP BY CONTACT_DATE
    ORDER BY CONTACT_DATE

In [ ]:
non_card_date

## CHECK CELL CODES AND CELL NAMES

In [ ]:
%%sql cells <<
    SELECT CELL_CODE, CELL_NAME
    FROM DM.CELL
    WHERE campaign_code = :campaign_code
    GROUP BY CELL_CODE, CELL_NAME
    ORDER BY CELL_CODE;

In [ ]:
cells

- Card

In [ ]:
cell_codes = %sql SELECT CELL_CODE, COUNT(*) AS COUNT FROM DW.CONTACT \
                  WHERE CAMPAIGN_CODE = :campaign_code GROUP BY CELL_CODE
try:
    cell_codes_cellnames=cell_codes.set_index('cell_code').join(cells.set_index('cell_code'))
    display(cell_codes_cellnames)
except:
    pass

- NonCard

In [ ]:
cell_codes = %sql SELECT CELL_CODE, COUNT(*) AS COUNT FROM CS.NON_CARD_CONTACT \
                  WHERE CAMPAIGN_CODE = :campaign_code GROUP BY CELL_CODE
try:
    cell_codes_cellnames=cell_codes.set_index('cell_code').join(cells.set_index('cell_code'))
    display(cell_codes_cellnames)
except:
    pass

## CHECK FEATURED PRODUCT AND BRAND/CATEGORY TABLES - DONOTDELETE

In [ ]:
%sql SELECT COUNT(*) FROM CS.BRANDS_PCA_BRAND_OR_CATEGORY_TABLE_DONOTDELETE WHERE CAMPAIGN_CODE = :campaign_code

In [ ]:
%sql SELECT UNIQUE(CONTACT_DATE) FROM CS.BRANDS_PCA_BRAND_OR_CATEGORY_TABLE_DONOTDELETE WHERE CAMPAIGN_CODE = :campaign_code

In [ ]:
%sql SELECT COUNT(*) FROM CS.BRANDS_PCA_FEATURED_PRODUCT_TABLE_DONOTDELETE WHERE CAMPAIGN_CODE = :campaign_code

In [ ]:
%sql SELECT UNIQUE(CONTACT_DATE) FROM CS.BRANDS_PCA_FEATURED_PRODUCT_TABLE_DONOTDELETE WHERE CAMPAIGN_CODE = :campaign_code

## CHECK ADOBE TABLES - DONOTDELETE

In [ ]:
campaign = %sql SELECT * FROM DWS.AC_CAMPAIGN WHERE UNICA_CAMPAIGN_CODE = :campaign_code 
print('DWS.AC_CAMPAIGN')
display(campaign)

send = %sql SELECT * FROM DWS.AC_SEND WHERE UNICA_CAMPAIGN_CODE = :campaign_code 
print('DWS.AC_SEND')
display(send)

open = %sql SELECT * FROM DWS.AC_OPEN WHERE UNICA_CAMPAIGN_CODE = :campaign_code 
print('DWS.AC_OPEN')
display(open)

click = %sql SELECT * FROM DWS.AC_CLICK WHERE UNICA_CAMPAIGN_CODE = :campaign_code 
print('DWS.AC_CLICK')
display(click)

unsubscribe = %sql SELECT * FROM DWS.AC_UNSUBSCRIBE WHERE UNICA_CAMPAIGN_CODE = :campaign_code
print('DWS.AC_UNSUBSCRIBE')
display(unsubscribe)

## CHECK PCA TABLES - can DELETE FROM

In [ ]:
tracker = %sql SELECT * FROM CS.PCA_CAMPAIGN_TRACKER WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMPAIGN_TRACKER')
display(tracker)

country_id = %sql SELECT * FROM CS.PCA_CAMPAIGN_ID_COUNTRY WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMPAIGN_ID_COUNTRY')
display(country_id)

landing = %sql SELECT * FROM CS.PCA_CAMPAIGN_LANDING WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMPAIGN_LANDING')
display(landing)
    
details = %sql SELECT * FROM CS.PCA_CAMPAIGN_DETAILS WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMPAIGN_DETAILS')
display(details)

camp_id = %sql SELECT * FROM CS.PCA_CAMP_ID WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMP_ID')
display(camp_id)

CH_match = %sql SELECT * FROM CS.PCA_CHEETAH_CH_MATCH WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CHEETAH_CH_MATCH')
display(CH_match)

trans_agg = %sql SELECT * FROM CS.PCA_TRANSACTION_AGG WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_TRANSACTION_AGG')
display(trans_agg)

email_eng = %sql SELECT * FROM CS.PCA_EMAIL_ENGAGEMENT WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_EMAIL_ENGAGEMENT')
display(email_eng)
            
email_response = %sql SELECT * FROM CS.PCA_EMAIL_RESPONSE_AGG WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_EMAIL_RESPONSE_AGG')
display(email_response)

cust_summary = %sql SELECT * FROM CS.PCA_CUST_SUMMARY WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CUST_SUMMARY')
display(cust_summary)

### BRAND AND FP

In [ ]:
camp_brand = %sql SELECT * FROM CS.PCA_CAMPAIGN_BRAND WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_CAMPAIGN_BRAND')
display(camp_brand)
    
brand_trans = %sql SELECT * FROM CS.PCA_BRAND_TRANSACTIONS WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_BRAND_TRANSACTIONS')
display(brand_trans)
    
brand_trans_segment = %sql SELECT * FROM CS.PCA_BRAND_TRANSACTIONS_SEGMENTATION WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_BRAND_TRANSACTIONS_SEGMENTATION')
display(brand_trans_segment)
    
bc_product_match = %sql SELECT * FROM CS.PCA_BRAND_OR_CATEGORY_PRODUCT_TABLE_MATCHED WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_BRAND_OR_CATEGORY_PRODUCT_TABLE_MATCHED')
display(bc_product_match)
    
fp_matched = %sql SELECT * FROM CS.PCA_FEATURED_PRODUCT_TABLE_MATCHED WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_FEATURED_PRODUCT_TABLE_MATCHED')
display(fp_matched) 

### NON-CARD TABLES

In [ ]:
email_ch_non_card = %sql SELECT * FROM CS.PCA_EMAIL_CH_NON_CARD_ONLY WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_EMAIL_CH_NON_CARD_ONLY')
display(email_ch_non_card)

trans_non_card = %sql SELECT * FROM CS.PCA_TRANSACTION_AGG_NON_CARD WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_TRANSACTION_AGG_NON_CARD')
display(trans_non_card)

### FINAL PCA TABLES

In [ ]:
offer_final_segment = %sql SELECT * FROM CS.PCA_OFFER_PERFORMANCE_FINAL_SEGMENTATION WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_OFFER_PERFORMANCE_FINAL_SEGMENTATION')
display(offer_final_segment)

final_segment = %sql SELECT * FROM CS.PCA_FINAL_TABLE_SEGMENTATION WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_FINAL_TABLE_SEGMENTATION')
display(final_segment)

final_segment_DM = %sql SELECT * FROM CS.PCA_DM_FINAL_TABLE_SEGMENTATION WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_DM_FINAL_TABLE_SEGMENTATION')
display(final_segment_DM)

final = %sql SELECT * FROM CS.PCA_FINAL_TABLE WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_FINAL_TABLE')
display(final)

final_DM = %sql SELECT * FROM CS.PCA_DM_FINAL_TABLE WHERE CAMPAIGN_CODE = :campaign_code
print('CS.PCA_DM_FINAL_TABLE')
display(final_DM)

In [ ]:
%%sql
    SELECT
    CAMPAIGN_CODE,
    CAMP_ID,
    CH_SEND_DATE,
    SUM(MAILED) AS MAILED,
    SUM(CONTROL) AS CONTROL,
    SUM(MAILED_SALES) AS MAILED_SALES,
    SUM(MAILED_SALES_FP) AS MAILED_SALES_FP,
    SUM(MAILED_SALES_BOC) AS MAILED_SALES_BOC
    FROM
    CS.PCA_FINAL_TABLE
    WHERE
    CAMPAIGN_CODE = :campaign_code
    GROUP BY
    CAMPAIGN_CODE,
    CAMP_ID,
    CH_SEND_DATE
    ORDER BY
    3 DESC

In [ ]:
%%sql
    SELECT
    CAMPAIGN_CODE,
    LAND_DATE,
    SUM(MAILED) AS MAILED,
    SUM(CONTROL) AS CONTROL,
    SUM(MAILED_SALES) AS MAILED_SALES,
    SUM(MAILED_SALES_FP) AS MAILED_SALES_FP,
    SUM(MAILED_SALES_BOC) AS MAILED_SALES_BOC
    FROM
    CS.PCA_DM_FINAL_TABLE
    WHERE
    CAMPAIGN_CODE = :campaign_code
    GROUP BY
    CAMPAIGN_CODE,
    LAND_DATE
    ORDER BY
    3 DESC

### EXTRA CHECK - loaded PCA info

#### FIND PRODUCTS 

In [ ]:
%%sql PRODUCTS <<
    SELECT * FROM CS.BRANDS_PCA_BRAND_OR_CATEGORY_TABLE_DONOTDELETE
    WHERE CAMPAIGN_CODE = :campaign_code
    AND CONTACT_DATE = :contact_date

In [ ]:
PRODUCTS

In [ ]:
item_codes = PRODUCTS['item_code'].unique().tolist()
print('There are',len(item_codes),'brand/category products associated with this campaign.')

In [ ]:
example_item_code = item_codes[0]
example_item_code 

#### FIND brand_code and item_hierarchy4_number associated to PCA tables

In [ ]:
BC_codes_df = %sql SELECT BRAND_CODE, ITEM_CODE, ITEM_DESCRIPTION, ITEM_HIERARCHY4_NUMBER FROM DW.MDM_ITEM WHERE ITEM_CODE = :example_item_code
    
PCA_brand_code = BC_codes_df['brand_code'][0]
PCA_item_hierarchy4_number = BC_codes_df['item_hierarchy4_number'][0]

print('brand_code associated to products loaded to PCA table is:',PCA_brand_code)
print('item_hierarchy4_number associated to products loaded to PCA table is:',PCA_item_hierarchy4_number)

#### FIND fp_codes 

In [ ]:
%%sql FP_df <<
    SELECT * FROM CS.BRANDS_PCA_FEATURED_PRODUCT_TABLE_DONOTDELETE
    WHERE CAMPAIGN_CODE = :campaign_code
    AND CONTACT_DATE = :contact_date

In [ ]:
FP_df

In [ ]:
fp_codes = FP_df['item_code'].unique().tolist()
print('There are',len(fp_codes),'item codes loaded to the PCA table.')

#### CHECK FEATURED PRODUCTS ITEM CODES

In [ ]:
if isinstance(fp_codes,int)==True:
    fp_check=f'''SELECT ITEM_CODE, ITEM_DESCRIPTION 
            FROM DW.MDM_ITEM WHERE ITEM_CODE = {fp_codes}'''
else:
    fp_check='SELECT ITEM_CODE, ITEM_DESCRIPTION FROM DW.MDM_ITEM WHERE \
            ITEM_CODE IN (%s)'%','.join('%s'%x for x in fp_codes)
    
%sql $fp_check